In [ ]:
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   -------------- ------------------------- 1.0/2.8 MB 8.5 MB/s eta 0:00:01
   -------------------------- ------------- 1.8/2.8 MB 3.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 3.2 MB/s eta 0:00:00


In [1]:
!pip install hmmlearn

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from hmmlearn import hmm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [3]:
def read_fasta(file_path, filter_header=None):
    sequences = ''
    for record in SeqIO.parse(file_path, "fasta"):
        if filter_header and filter_header not in record.id:
            continue
        sequences = record.seq
    return sequences

def read_centro_telo(file_path_centro, file_path_telo, file_path_cpg):
    centro = pd.read_csv(file_path_centro)
    centro.drop('#"bin"', axis=1, inplace=True)
    
    telo = pd.read_csv(file_path_telo)
    telo.drop('#bin', axis=1, inplace=True)
    telo = telo[telo['type'] == 'telomere']
    
    cpg = pd.read_csv(file_path_cpg)
    cpg.drop('#"bin"', axis=1, inplace=True)
    
    return centro, telo, cpg

def check_overlap(centro, telo, cpg):
    telo.sort_values('chromStart', inplace=True)
    # print("Starting Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[0, 0]},{telo[['chromStart', 'chromEnd']].iloc[0, 1]})')
    # print("Ending Telomere")
    # print(f'({telo[['chromStart', 'chromEnd']].iloc[1, 0]},{telo[['chromStart', 'chromEnd']].iloc[1, 1]})')
    
    start_telo = telo[['chromStart', 'chromEnd']].iloc[0, 1]
    end_telo = telo[['chromStart', 'chromEnd']].iloc[1, 0]

    if len(cpg[cpg['chromStart'] < start_telo]) > 0 or len(cpg[cpg['chromEnd'] > end_telo]) > 0:
        print("There is a CpG island overlapping with Telomeric regions.")
    else:
        print("There is NO overlapping CpG island with Telomeric regions.")
        
    centro.sort_values('chromStart', inplace=True)
    min_centro = centro[['chromStart','chromEnd']].iloc[0,0]
    max_centro = centro[['chromStart','chromEnd']].iloc[len(centro)-1, 1]

    if len(cpg[(cpg['chromStart'] > min_centro) & (cpg['chromEnd'] < max_centro)]) > 0:
        print("Do a manual inspection because there is a CpG island in between the Centromeric regions.")
    else:
        print("There is NO overlapping CpG island with Centromeric regions.")

In [4]:
seq = read_fasta('./GRCh38.p14.genome.fa', 'chr18')
centro18_df, telo18_df, cpg18_df = read_centro_telo('./chr18/centromeres.csv','./chr18/telomeres.csv', './chr18/cpg.csv')

In [3]:
check_overlap(centro18_df, telo18_df, cpg18_df)

In [7]:
def check_in_start_end(i, index, start_end, in_):
    if index < len(start_end):
        if not in_ and i >= start_end[index]:
            return True, index + 1
        if in_ and i >= start_end[index]:
            return False, index + 1
    return in_, index    

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [10]:
# check regions for ith base
in_cpg = False
cpg_index = 0

in_centro = False
centro_index = 0

in_telo = False
telo_index = 0

# check regions for (i+1)th base
in_cpg_next = False
cpg_index_next = 0

in_centro_next = False
centro_index_next = 0

in_telo_next = False
telo_index_next = 0

# flatten the regions
telo_start_end = telo18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
centro_start_end = centro18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()
cpg_start_end = cpg18_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()

# Format: A-, G-, C-, T-, A+, G+, C+, T+
init_probs = np.zeros(8) # data structure for initial probabilities
tran_probs = np.zeros((8,8)) # data structure for transition probabilities

# base to index
init_dict = {
    'A-': 0,
    'G-': 1,
    'C-': 2,
    'T-': 3,
    'A+': 4,
    'G+': 5,
    'C+': 6,
    'T+': 7,
}

# linearly traverse the sequence
for i in range(len(seq)):
    # check if in telomeric region
    in_telo, telo_index = check_in_start_end(i, telo_index, telo_start_end, in_telo)
    if in_telo:
        continue

    # check if in centromeric region
    in_centro, centro_index = check_in_start_end(i, centro_index, centro_start_end, in_centro)
    if in_centro:
        continue

    # check if in CpG island
    in_cpg, cpg_index = check_in_start_end(i, cpg_index, cpg_start_end, in_cpg)

    # avoid ambiguous bases N
    if seq[i] == 'N':
        continue

    # counting the frequency of each Markov States
    base_i = seq[i] + ('+' if in_cpg else '-')
    init_probs[init_dict[base_i]] += 1

    if i + 1 < len(seq):
        # check if next index in telomeric region
        in_telo_next, telo_index_next = check_in_start_end(i + 1, telo_index_next, telo_start_end, in_telo_next)
        if in_telo_next:
            continue

        # check if next base in centromeric region
        in_centro_next, centro_index_next = check_in_start_end(i + 1, centro_index_next, centro_start_end, in_centro_next)
        if in_centro_next:
            continue

        # check if next base in CpG island
        in_cpg_next, cpg_index_next = check_in_start_end(i + 1, cpg_index_next, cpg_start_end, in_cpg_next)

        # avoid ambiguous bases N
        if seq[i+1] == 'N':
            continue

        base_i_next = seq[i + 1] + ('+' if in_cpg_next else '-')
        tran_probs[init_dict[base_i_next], init_dict[base_i]] += 1

In [14]:
final_init_probs = init_probs/init_probs.sum()
final_init_probs

array([0.29997011, 0.19703748, 0.19670899, 0.30027983, 0.00093958,
       0.00205016, 0.00206689, 0.00094697])

In [13]:
final_tran_probs = tran_probs/tran_probs.sum(axis=1)[:, np.newaxis]
final_tran_probs

array([[3.36673990e-01, 1.95908309e-01, 2.39228107e-01, 2.28186162e-01,
        0.00000000e+00, 3.43276126e-06, 0.00000000e+00, 0.00000000e+00],
       [3.47248782e-01, 2.44139710e-01, 4.30627657e-02, 3.65536322e-01,
        0.00000000e+00, 1.24203267e-05, 0.00000000e+00, 0.00000000e+00],
       [2.54927379e-01, 2.02469356e-01, 2.44062515e-01, 2.98531777e-01,
        0.00000000e+00, 8.97388884e-06, 0.00000000e+00, 0.00000000e+00],
       [2.67780742e-01, 1.67633343e-01, 2.27956537e-01, 3.36624168e-01,
        0.00000000e+00, 5.21063363e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.99077698e-01, 3.56594885e-01, 3.58744075e-01, 8.55833416e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.89002316e-01, 3.56557190e-01, 2.92117022e-01, 1.62323473e-01],
       [6.34074381e-04, 8.41119077e-04, 1.30696964e-03, 5.11141593e-04,
        1.24278579e-01, 3.47272186e-01, 3.57223272e-01, 1.

In [11]:
# sanity check for initial probabilities
print("Total should be 1")
print(final_init_probs.sum())
# sanity check for transition probabilities
print("All elements must total to 1")
print(final_tran_probs.sum(axis=1))

1.0

In [15]:
# Checkpoint for transition probabilities
np.savetxt("transition_probs.csv", final_tran_probs, delimiter=",")
# Checkpoint forr initial probabilities
np.savetxt("initial_probs.csv", final_init_probs, delimiter=",")

### Extract regions of chr19 WITHOUT telomere and centromere

In [12]:
seq_test = read_fasta('./GRCh38.p14.genome.fa', 'chr19')
centro19_df, telo19_df, cpg19_df = read_centro_telo('./chr19/centromeres.csv','./chr19/telomeres.csv', './chr19/cpg.csv')

In [13]:
check_overlap(centro19_df, telo19_df, cpg19_df)

There is NO overlapping CpG island with Telomeric regions.
There is NO overlapping CpG island with Centromeric regions.


In [46]:
# Extract 2 subsequences from chr19 such that
# 10000-first centromere
# skip all centromere regions 
# end of last centromere - (end- 10000)

# Extract start of the first centromere and end of last centromere
start_centro = centro19_df['chromStart'][0]
end_centro = centro19_df['chromEnd'].iloc[-1]

# Extract end of the first telomere and start of the last telomere
end_first_telo = telo19_df['chromEnd'].iloc[0]
start_last_telo = telo19_df['chromStart'].iloc[-1]

# Extract 2 subsequences 
seq_test_1 = seq_test[end_first_telo: start_centro] # end of first telomere until start of first centromere
seq_test_2 = seq_test[end_centro: start_last_telo]  # end of last centromere until start of last telomere

In [48]:
# search for first index without N
def find_first_valid_index(seq):
    for i, base in enumerate(seq):
        if base != 'N':
            return i
    return -1

# search for first index with N
def find_first_invalid_index(seq):
    for i, base in enumerate(seq):
        if base == 'N':
            return i
    return -1

def find_valid_subsequence(seq):
    start_index = find_first_valid_index(seq)
    if start_index == -1:
        return 0, 0, ''

    end_index = find_first_invalid_index(seq[start_index:])
    if end_index == -1:
        return start_index, len(seq), seq[start_index:]

    end_index += start_index
    return start_index, end_index, seq[start_index:end_index]


start_index_1, end_index_1, valid_subsequence_1 = find_valid_subsequence(seq_test_1)
start_index_2, end_index_2, valid_subsequence_2 = find_valid_subsequence(seq_test_2)

with open('seq_test_1_withoutN.fasta', 'w') as f:
    f.write(f'>seq_test_1_withoutN\tindex {end_first_telo + start_index_1}:{end_first_telo+ end_index_1}\n{valid_subsequence_1}\n')
with open('seq_test_2_withoutN.fasta', 'w') as f:
    f.write(f'>seq_test_2_withoutN\tindex {end_centro + start_index_2}:{end_centro+ end_index_2}\n{valid_subsequence_2}\n')

GATCACAGAGGCTGGGCTGCTCCCCACCCTCTGCACACCTCCTGCTTCTAACAGCAGAGCTGCCAGGCCAGGCCCTCAGGCAAGGGCTCTGAAGTCAGGG
GATCCTTTACACAGAACAGATTTGAAACACTCTTTTTGTGGAATTTCCAAGTGGAGATTTCAGCCGATTTGAAGTCAATCGTAGAAAAGGAGATATCTTC


### Build HMM

In [5]:
states = ['A-', 'G-', 'C-', 'T-', 'A+', 'G+', 'C+', 'T+']
n_states = len(states)

In [6]:
transition_probs = np.loadtxt("transition_probs.csv", delimiter=",")
initial_probs = np.loadtxt("initial_probs.csv", delimiter=",")
emit_probs = np.concatenate([np.eye(4), np.eye(4)])

In [ ]:
model = hmm.CategoricalHMM(n_components=n_states, verbose=True)
model.startprob_= initial_probs
model.emissionprob_ = emit_probs
model.transmat_ = transition_probs

In [10]:
def get_gene_sequence(filename):
    file = read_fasta(filename)
    gene = str(file)            
    out = []
    
    for char in gene :
        if char == "A":
            out.append([0])
        elif char == "G":
            out.append([1])
        elif char == "C":
            out.append([2])
        else:
            out.append([3])
            
    return np.array(out)

In [11]:
test_sequence_np_1 = get_gene_sequence("seq_test_1_withoutN.fasta")

In [ ]:
logprob_1, output_bases_1 = model.decode(test_sequence_np_1, algorithm="viterbi")

In [ ]:
out_1 = "%s"%" ".join([states[x][-1] for x in output_bases_1])

In [78]:
with open ("output_1.txt", "w") as f:
    f.write(out_1)

In [11]:
test_sequence_np_2 = get_gene_sequence("seq_test_2_withoutN.fasta")

In [12]:
logprob_2, output_bases_2 = model.decode(test_sequence_np_2, algorithm="viterbi")

In [15]:
out_2 = "%s"%" ".join([states[x][-1] for x in output_bases_2])

In [16]:
with open ("output_2.txt", "w") as f:
    f.write(out_2)

# Confusion Matrix

In [ ]:
_, _, cpg19_df = read_centro_telo('./chr19/centromeres.csv','./chr19/telomeres.csv', './chr19/cpg.csv')
cpg_start_end = cpg19_df.sort_values('chromStart')[['chromStart', 'chromEnd']].to_numpy().flatten()

In [13]:
output_1 = np.loadtxt("output_1.txt", delimiter=" ", dtype=str)

In [122]:
# TODO: read the fasta header
start_index_1 = 60_000
end_index_1 = 24_448_980

In [147]:
def get_ground_truth(start_index, end_index, cpg_start_end):
    ground_truth = []
    starting_region = '-'

    # determine if the start index is in a non-CpG or CpG island
    for i, each in enumerate(cpg_start_end):
        if start_index < each:
            if i % 2:
                starting_region = '+'
            break
            
    # [starting_region, opposite]
    annot = [starting_region, '-' if starting_region == '+' else '+']
    
    # generate the ground truth
    for j, k in enumerate(cpg_start_end[i:]):
        if j == 0:
            ground_truth = np.full(each-start_index, starting_region)
        else:
            if k > end_index:
                ground_truth = np.append(ground_truth, np.full(end_index-cpg_start_end[j-1], annot[j % 2]))
                break
            else:
                ground_truth = np.append(ground_truth, np.full(k-cpg_start_end[j-1], annot[j % 2]))
    return ground_truth, annotation

def get_performance_metrics(confusion_mat):
    TN, FP = confusion_mat[0]
    FN, TP = confusion_mat[1]
    
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

0 70343 24448980
1 71344 24448980
2 267085 24448980
3 267660 24448980
4 290594 24448980
5 292249 24448980
6 295114 24448980
7 295354 24448980
8 306214 24448980
9 306681 24448980
10 308413 24448980
11 308930 24448980
12 310258 24448980
13 310582 24448980
14 311948 24448980
15 312224 24448980
16 344164 24448980
17 344839 24448980
18 345260 24448980
19 345590 24448980
20 360071 24448980
21 361949 24448980
22 375830 24448980
23 376094 24448980
24 384250 24448980
25 384464 24448980
26 387964 24448980
27 388350 24448980
28 401970 24448980
29 402280 24448980
30 403553 24448980
31 403864 24448980
32 406131 24448980
33 406611 24448980
34 407204 24448980
35 409511 24448980
36 418898 24448980
37 419191 24448980
38 422147 24448980
39 422629 24448980
40 426321 24448980
41 427819 24448980
42 429260 24448980
43 429581 24448980
44 429995 24448980
45 430323 24448980
46 436540 24448980
47 436742 24448980
48 440535 24448980
49 440918 24448980
50 450801 24448980
51 451376 24448980
52 452130 24448980
53 45

In [152]:
# generate the confusion matrix
ground_truth_1 = get_ground_truth(start_index_1, end_index_1, cpg_start_end)
confusion_mat = confusion_matrix(ground_truth_1, output, labels=annot)
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_mat,
                              display_labels=annot)
disp.plot()
plt.show()

['-', '+']


In [155]:
get_performance_metrics(confusion_mat)

Precision: 0.07
Recall: 0.99
Accuracy: 0.54


In [ ]:
# TODO add posterior plus another subseq